# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://linkedin.com/in/rameasy")
links

['https://www.linkedin.com/legal/cookie-policy',
 'https://www.linkedin.com/mypreferences/g/guest-cookies',
 '#main-content',
 '/?trk=public_profile_nav-header-logo',
 'https://www.linkedin.com/top-content?trk=public_profile_guest_nav_menu_topContent',
 'https://www.linkedin.com/pub/dir/+/+?trk=public_profile_guest_nav_menu_people',
 'https://www.linkedin.com/learning/search?trk=public_profile_guest_nav_menu_learning',
 'https://www.linkedin.com/jobs?trk=public_profile_guest_nav_menu_jobs',
 'https://www.linkedin.com/games?trk=public_profile_guest_nav_menu_games',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_nav_upsell&trk=public_profile_guest_nav_menu_windows',
 'https://www.linkedin.com/login?session_redirect=https%3A%2F%2Fuk%2Elinkedin%2Ecom%2Fin%2Frameasy&fromSignIn=true&trk=public_profile_nav-header-signin',
 'https://www.linkedin.com/signup/public-profile-join?vieweeVanityName=rameasy&session_redirect=https%3A%2F%2Fuk.linkedin.com%2Fin%2Frameasy&trk=public

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://linkedin.com/in/rameasy"))


Here is the list of links on the website https://linkedin.com/in/rameasy -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://www.linkedin.com/legal/cookie-policy
https://www.linkedin.com/mypreferences/g/guest-cookies
#main-content
/?trk=public_profile_nav-header-logo
https://www.linkedin.com/top-content?trk=public_profile_guest_nav_menu_topContent
https://www.linkedin.com/pub/dir/+/+?trk=public_profile_guest_nav_menu_people
https://www.linkedin.com/learning/search?trk=public_profile_guest_nav_menu_learning
https://www.linkedin.com/jobs/jobs-in-enfield?trk=public_profile_guest_nav_menu_jobs
https://www.linkedin.com/games?trk=public_profile_guest_nav_menu_games
ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_nav_upsell&trk=public_profile_guest_nav_menu_windows
https://www.linkedin.co

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://linkedin.com/in/rameasy")

{'links': [{'type': 'about page',
   'url': 'https://about.linkedin.com?trk=public_profile_v3_desktop_footer-about'},
  {'type': 'company page',
   'url': 'https://uk.linkedin.com/company/natwest-group?trk=public_profile_topcard-current-company'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/udemy?trk=public_profile_profile-section-card_image-click'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/oracle?trk=public_profile_profile-section-card_image-click'},
  {'type': 'careers page',
   'url': 'https://www.linkedin.com/jobs/jobs-in-london?trk=public_profile_guest_nav_menu_jobs'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [10]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [11]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
1 day ago
•
286k
•
787
deepseek-ai/DeepSeek-OCR
Updated
6 days ago
•
1.34M
•
2.26k
meituan-longcat/LongCat-Video
Updated
1 day ago
•
660
•
209
PaddlePaddle/PaddleOCR-VL
Updated
1 day ago
•
22k
•
1.17k
tencent/HunyuanWorld-Mirror
Updated
6 days ago
•
18.3k
•
396
Browse 1M+ models
Spaces
Running
15.5k
15.5k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
on
Zero
351
351
DeepSeek OCR Demo
🆘
An interactive demo for the DeepSeek-OCR model.
Running
2.15k
2.15k
Wan2.2 Animate
👁
Wan2.2 Animate
Runnin

In [12]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [14]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nMiniMaxAI/MiniMax-M2\nUpdated\n1 day ago\n•\n286k\n•\n787\ndeepseek-ai/DeepSeek-OCR\nUpdated\n6 days ago\n•\n1.34M\n•\n2.26k\nmeituan-longcat/LongCat-Video\nUpdated\n1 day ago\n•\n660\n•\n209\nPaddlePaddle/PaddleOCR-VL\nUpdated\n1 day ago\n•\n22k\n•\n1.17k\ntencent/HunyuanWorld-Mirror\nUpdated\n6 days ago\n•\n18.3k\n•\n396\nBrowse 1M+ models\nSpaces\nRunning\n15.5k\n15.5k\nDeepSite v3\n🐳\nG

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the leading AI community and collaboration platform dedicated to building the future of machine learning. Founded on the mission to democratize good machine learning "one commit at a time," Hugging Face provides a vibrant ecosystem where thousands of developers, researchers, and enterprises come together to create, share, and innovate on open-source models, datasets, and applications.

Their platform hosts over **1 million models** and **250,000+ datasets** across a range of modalities including text, image, video, audio, and even 3D data. Hugging Face serves not just as a repository but as a creative space where users can build and demonstrate AI applications through interactive "Spaces," providing an accelerated route from development to deployment.

---

## What Hugging Face Offers

### Community and Collaboration
- A thriving community of over **65,000+ followers** engaged in sharing, updating, and innovating ML models and datasets.
- Open ecosystem supporting collaboration across individuals, teams, and enterprises.
- Regular updates on cutting-edge research and tools, with active contribution from community members and researchers globally.

### Hugging Face Hub
- Centralized repository hosting:
  - Over **1 million models** covering state-of-the-art NLP, computer vision, and multimodal models.
  - Extensive, curated datasets to train, validate, and benchmark models.
- Supports all ML modalities to build versatile and inclusive AI solutions.
- Allows build and showcase of ML projects to build professional portfolios and profiles.

### Enterprise & Compute Solutions
- Paid Compute and Enterprise-grade offerings tailored for business scalability.
- Robust platform enables teams to collaborate with enhanced security, access controls, and support.
- Accelerates product development with cloud-based infrastructure and deployment tools.

---

## Company Culture

Hugging Face fosters a culture rooted in **open science, ethics, and community-driven progress**. The organization believes in transparency, sharing, and democratizing machine learning tools to empower engineers, scientists, and AI users to build an open and ethical AI future together.

- Encourages contributions from all levels of expertise.
- Values innovation, learning, and continuous improvement.
- Supports a distributed and diverse team of nearly 200 members passionate about AI and machine learning.

---

## Customers and Impact

Hugging Face's platform is trusted by:
- Individual ML engineers and researchers.
- Academic institutions.
- AI startups and technology companies.
- Large enterprises seeking to safely and efficiently deploy AI at scale.

Popular models like DistilBERT (a lightweight version of BERT) originated here, showcasing Hugging Face's influence on democratizing advanced NLP technology.

---

## Careers at Hugging Face

Join a fast-growing company at the forefront of the AI revolution! Hugging Face offers a dynamic environment for engineers, researchers, and product builders who want to:

- Collaborate on impactful open-source machine learning projects.
- Be part of a vibrant, mission-driven AI community.
- Work with state-of-the-art models and datasets on a global platform.
- Thrive in a culture that values openness, ethical AI, and continuous learning.

Visit their careers page to explore open positions and become part of the team shaping the future of machine learning.

---

## Brand and Identity

- The company brand is represented by vibrant and approachable assets emphasizing openness and creativity.
- Official colors include bright yellow (#FFD21E), orange (#FF9D00), and neutral gray (#6B7280).
- Logos and brand materials are freely available for community and partner use, reflecting the collaborative ethos.

---

## How to Get Started

- Explore their platform at [huggingface.co](https://huggingface.co)
- Browse models, datasets, and community apps.
- Sign up to create your own spaces and share your work.
- Join the vibrant community shaping AI innovation globally.

---

**Hugging Face — The AI community building the future.**  
Empower your AI projects with open source, collaboration, and innovation.  

---

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant and fast-growing AI and machine learning community dedicated to advancing the future of open and ethical AI. It serves as a collaboration platform where machine learning engineers, scientists, and enthusiasts from around the world create, discover, and share machine learning models, datasets, and applications.

The Hugging Face Hub is the central place for hosting and collaborating on over **1 million models**, **250k+ datasets**, and **400k+ applications** spanning multiple modalities including text, images, video, audio, and 3D.

---

## What Hugging Face Offers

- **Models:** Browse and explore a vast collection of AI models, updated frequently by a diverse community. Popular trending models include image generators, video editors, and language models.
  
- **Datasets:** Access a rich catalog of datasets ranging from natural language prompts to persona datasets and beyond, enabling rapid model training and experimentation.
  
- **Spaces:** Deploy, share, and run AI applications on Hugging Face's platform, including innovative tools such as video generation from images, AI toolkits, and interactive applications.

- **Compute:** Easily accelerate your machine learning workloads with optimized inference endpoints and GPU-backed applications starting as low as $0.60/hour.

- **Enterprise Solutions:** Hugging Face provides tailored enterprise offerings to help organizations harness AI technology securely and effectively at scale.

---

## Company Culture

Hugging Face fosters an **open, collaborative, and ethical AI community** — empowering developers and researchers to work together transparently and inclusively. The culture emphasizes:

- **Community-driven innovation:** Encouraging open-source contributions and sharing knowledge.
- **Ethical AI development:** Committed to building responsible AI systems for the benefit of all.
- **Diversity and inclusion:** Supporting a broad and global community in building AI tools and resources.
- **Learning and growth:** Offering a platform for users to build their machine learning portfolios and grow their expertise.

---

## Customers & Community

Hugging Face serves a global community that includes:

- Machine learning researchers and practitioners
- Developers building innovative AI applications
- Enterprises seeking scalable AI solutions
- AI enthusiasts and learners who want to experiment and grow their skills

With over a million models and hundreds of thousands of datasets and applications, the platform connects users with cutting-edge AI technologies from companies like Tencent, NVIDIA, Facebook, and OpenAI — showcasing the power of open collaboration.

---

## Careers at Hugging Face

Interested in joining Hugging Face? The company values talented individuals passionate about AI, open-source technologies, and community building. Opportunities include roles in:

- Machine learning research & engineering
- Software development
- Cloud infrastructure and compute optimization
- Community management and developer relations
- Enterprise solutions & sales

At Hugging Face, you will work at the forefront of AI innovation while contributing to a global, open, and ethical AI movement.

---

## Get Involved

- **Explore AI models and datasets:** Visit [huggingface.co](https://huggingface.co) to browse millions of resources.
- **Build and share your own models and apps:** Create your portfolio in an open, collaborative environment.
- **Accelerate your projects:** Utilize Hugging Face’s Compute and Enterprise services.
- **Join the community:** Engage with thousands of practitioners working toward the future of AI.

---

### Contact & Follow

- Website: [https://huggingface.co](https://huggingface.co)
- Join the community to learn, collaborate, and innovate in AI!

---

*Hugging Face — Empowering the next generation of machine learning for a better, open future.*

In [19]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


# Hugging Face Brochure

## About Hugging Face
Hugging Face is the AI community and collaboration platform at the forefront of the machine learning revolution. It provides an open, ethical, and inclusive space where machine learning engineers, scientists, and enthusiasts come together to share, explore, and build cutting-edge models, datasets, and applications. Hugging Face is home to one of the fastest-growing AI communities and hosts over 1 million models, 250,000+ datasets, and 400,000+ applications spanning modalities like text, image, video, audio, and even 3D.

## What We Offer
- **Hugging Face Hub**: A central repository for hosting and sharing unlimited public models, datasets, and ML applications.
- **Spaces**: An easy-to-use platform to deploy and share interactive machine learning apps with the community.
- **Open Source Tools**: Access powerful ML libraries and frameworks that accelerate development and experimentation.
- **Enterprise Solutions**: Scalable AI platform with enterprise-grade security, single sign-on (SSO), granular access controls, private storage, audit logs, analytics dashboard, and advanced compute options like ZeroGPU scaling.

## Platform Highlights
- **Collaboration at Scale**: Enable teams and enterprises to securely collaborate and innovate with advanced resource management.
- **Explore & Build**: Browse through millions of models and datasets, discover trending AI applications, and deploy your own.
- **Portfolio Building**: Share your projects to gain visibility and build your machine learning profile within the vibrant HF community.
- **Accelerated Training & Inference**: Pay-for-use accelerated compute and managed inference services to power your ML workloads.

## Company Culture
Hugging Face is committed to building an open and ethical AI future through collaboration. It nurtures a passionate community-driven culture that values transparency, accessibility, and innovation. The company actively empowers the next generation of AI talent by providing collaborative tools and resources as well as fostering broad knowledge sharing.

## Who Uses Hugging Face?
- Machine learning researchers and engineers exploring cutting-edge AI architectures.
- Enterprises scaling AI across teams with secure and managed solutions.
- Developers and data scientists building applications spanning NLP, computer vision, audio processing, and more.
- Enthusiasts and learners building portfolios and learning collaboratively in an inclusive environment.

## Careers at Hugging Face
Hugging Face invites talented individuals who are passionate about AI to join its forward-thinking team. The company offers opportunities to work on open-source ML projects, tackle challenging scientific problems, and contribute to shaping the future of AI technology.

- Roles span engineering, research, community management, enterprise support, and beyond.
- Employees thrive in a culture that encourages continuous learning, innovation, and contributing to the global AI ecosystem.
- Careers at Hugging Face mean being part of an impactful mission: democratizing AI for all.

---

**Contact & Learn More:**

Visit [https://huggingface.co](https://huggingface.co)  
Explore the Hub, join the community, or inquire about enterprise solutions today.

_Hugging Face: The AI community building the future._

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>